In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error
%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
data=pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")
data.drop(columns=['id','url_legal','license'], inplace=True)


In [ ]:

dale=pd.read_csv('/kaggle/input/dalecsv/Dale.csv', header=None)
dale.columns=['words']
dale_list=dale.values.tolist()


In [ ]:
['arm'] in dale_list

In [ ]:
def split_word(list_word):
    new_word_list=[]
    for word in list_word:
        if '--' in word:
            new_list=word.split('--')
            for new_word in new_list:
                new_word_list.append(new_word)
        elif '-' in word:
            new_list=word.split('-')
            for new_word in new_list:
                new_word_list.append(new_word)
        elif '–' in word:
            new_list=word.split('–')
            for new_word in new_list:
                new_word_list.append(new_word)
        elif '—' in word:
            new_list=word.split('—')
            for new_word in new_list:
                new_word_list.append(new_word)
        
        
        else:
            new_word_list.append(word)

    return new_word_list

In [ ]:
def pancs_remove(list_word):    
    flag= True
    pancs=['.',',',';',':','"','\'','?','[',']','(',')','$','!','%','@','#','/','*','+','-','--','^','&','|','£','‘','°','–',"'s'"]
    new_word_list=[]
    while flag:
        for word in list_word:
            if len(word)==0:
                continue
            elif word==' ':
                continue
            elif word[-1] in pancs:
                new_word_list.append(word[:-1])
            elif word[0] in pancs:
                new_word_list.append(word[1:])
            else:
                new_word_list.append(word)
        check_list=[]
        for word in new_word_list:
            if len(word)==0:
                continue
            elif word==' ':
                continue
            if word[-1] in pancs:
                check_list.append(True)
            elif word[0] in pancs:
                check_list.append(True)
            else:
                check_list.append(False)
        checker=sum(check_list)
        if checker>0:
            list_word=new_word_list
            new_word_list=[]
            flag=True
        else:
            flag=False
           
    return new_word_list

In [ ]:
def listed_words(list_word):
    words=[]
    for word in list_word:
        if word.lower().isalpha():
            if word.lower() in words:
                continue
            else:
                words.append(word.lower())
    words2=words
    words=[]
    for word in words2:
        try:
            int(word)
            continue
        except:
            words.append(word)
    return words


In [ ]:
def sentence_length(paragraph):
#pancs=[',','.',';']
    sentence_len=[]
    sentence=[]
    paragraphs=paragraph.split('.')
    for sentences in paragraphs:
        sent=sentences.split(',')
        for senti in sent:
            sentence.append(senti)
    for sent in sentence:
        sentence_len.append(len(sent))
    return sum(sentence_len)/len(sentence_len)


In [ ]:
def difficult_words(words_list,check_list):
    dificult=[]
    for check in words_list:
        for word in check_list:
            if check==word:
                dificult.append(True)
                break
            elif check[:-1]==word:
                dificult.append(True)
                break
            elif word=="yourselves":
                dificult.append(False)
    return 1-(sum(dificult)/len(dificult))

In [ ]:
def difficult_word(words_list,check_list):
    dificult=[]
    for word in words_list:
        if list(word) in check_list:
            dificult.append(True)
            break
        elif list(word[:-1]) in check_list:
            dificult.append(True)
            break
        elif word=="yourselves":
            dificult.append(False)
    return 1-(sum(dificult)/len(dificult))

In [ ]:
data['excerpt_words']=0
data['words_count']=0
data['words_numbers']=0
data['difficult_words_percent']=0.0
data['sentence_length_mean']=0.0

In [ ]:
data['excerpt_words']=data['excerpt_words'].astype(object)

In [ ]:
for i in data.index:
    sent=(data['excerpt'][i].split())
    sent=split_word(sent)
    sent=pancs_remove(sent)
    sent=split_word(sent)
    words=listed_words(sent)
    data.at[i, 'excerpt_words']=words
    data['words_count'][i]=len(words)
    data['words_numbers'][i]=len(sent)

In [ ]:
difficult_word(data['excerpt_words'][1],dale_list)

In [ ]:
for i in data.index:
    data['sentence_length_mean'][i]=sentence_length(data['excerpt'][i])
    data['difficult_words_percent'][i]=difficult_words(data['excerpt_words'][i],dale.words)


In [ ]:
y=data.target
X=data.drop(columns=['excerpt','target','standard_error','excerpt_words','words_count','words_numbers'])


In [ ]:
data.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
data_model=DecisionTreeRegressor(max_leaf_nodes=16,random_state=0)
data_model.fit(X_train,y_train)
y_pred=data_model.predict(X_test)
mean_absolute_error(y_test,y_pred)

In [ ]:
data_model=DecisionTreeRegressor(max_leaf_nodes=16,random_state=0)
data_model.fit(X,y)


In [ ]:
data2=pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')

In [ ]:
data2.drop(columns=['url_legal','license'], inplace=True)


In [ ]:
data2['excerpt_words']=0
data2['words_count']=0
data2['words_numbers']=0
data2['difficult_words_percent']=0.0
data2['sentence_length_mean']=0.0

In [ ]:
data2['excerpt_words']=data2['excerpt_words'].astype(object)

In [ ]:
for i in data2.index:
    sent=(data2['excerpt'][i].split())
    sent=split_word(sent)
    sent=pancs_remove(sent)
    sent=split_word(sent)
    words=listed_words(sent)
    data2.at[i, 'excerpt_words']=words
    data2['words_count'][i]=len(words)
    data2['words_numbers'][i]=len(sent)

In [ ]:
for i in data2.index:
    data2['sentence_length_mean'][i]=sentence_length(data2['excerpt'][i])
    data2['difficult_words_percent'][i]=difficult_words(data2['excerpt_words'][i],dale.words)


In [ ]:
X2=data2.drop(columns=['id','excerpt','excerpt_words','words_count','words_numbers'])
X2.head()

In [ ]:
y_pred=data_model.predict(X2)

In [ ]:
submit = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')
submit.target = y_pred
submit

In [ ]:
submit.to_csv('submission.csv',index=False)